# Importy

In [1]:

from warnings import filterwarnings
filterwarnings('ignore')

from importlib import reload

### DataLoader
import DataLoader.xAPIConnector
reload(DataLoader.xAPIConnector)
from DataLoader.xAPIConnector import *

import DataLoader.DataLoader
reload(DataLoader.DataLoader)
from DataLoader.DataLoader import *

from DataLoader.config import user_id, pwd


### Data
import Data.SymbolParser
reload(Data.SymbolParser)
from Data.SymbolParser import parse_symbols


### TrendAnalysis
import TrendAnalysis.MACD
reload(TrendAnalysis.MACD)
from TrendAnalysis.MACD import MACD

import TrendAnalysis.MA
reload(TrendAnalysis.MA)
from TrendAnalysis.MA import MA

import TrendAnalysis.RSI
reload(TrendAnalysis.RSI)
from TrendAnalysis.RSI import RSI


### TechFunctions
from TechFunctions.Technicalities import position_encoding

# Właściwy kod

In [2]:
Symbols = parse_symbols()
for key, val in Symbols.items():
    print(f"{key}: {val}")

gold: ['NEM.US', 'AU.US', 'RGLD.US', 'CDE.US', 'SA.US', 'CMCL.US', 'IDR.US', 'CTGO.US', 'FTCO.US', 'USAU.US']
copper: ['SCCO.US', 'FCX.US', 'MTAL.US', 'IE.US']
aluminium: ['AA.US', 'CENX.US', 'CSTM.US', 'KALU.US']
silver: ['AG.US', 'MAG.US', 'AYASF.US', 'SVM.US']


In [3]:
symbols = Symbols['gold']
start, interval = '2020-01-01 00:00:00', '1D'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval, verbose=False)
data

[BŁĄD] Nie pobrano CDE.US
[BŁĄD] Nie pobrano SA.US
[BŁĄD] Nie pobrano CMCL.US
[BŁĄD] Nie pobrano IDR.US
[BŁĄD] Nie pobrano CTGO.US
[BŁĄD] Nie pobrano FTCO.US
[BŁĄD] Nie pobrano USAU.US


,NEM.US,AU.US,RGLD.US,GOLD,COPPER,ALUMINIUM,SILVER
Date,,,,,,,
2020-01-02,43.25,22.64,120.80,1528.80,6174.0,1772.0,17.969
2020-01-03,42.73,22.49,119.94,1548.92,6105.0,1789.0,18.007
2020-01-06,43.21,22.62,118.32,1565.17,6110.0,1801.0,18.103
2020-01-07,43.24,22.43,120.01,1571.09,6132.0,1785.0,18.356
2020-01-08,41.98,21.44,111.02,1555.92,6158.0,1769.0,18.103
...,...,...,...,...,...,...,...
2025-01-24,41.95,28.13,139.00,2770.92,9247.0,2633.0,30.622
2025-01-27,41.41,27.82,137.68,2740.44,9087.0,2606.0,30.170
2025-01-28,41.35,28.15,138.82,2762.99,9009.0,2571.0,30.395


In [ ]:
for s in [-1.0, 0.0, 1.0]:
    s_str = position_encoding[int(s)]
    
    model = MA(data)
    print(f"Sygnały {s_str} z modelu MA:\n{(model.Decision()==s).sum()}")

    model = MACD(data)
    print(f"Sygnały {s_str} z modelu MACD:\n{(model.Decision()==s).sum()}")

    model = RSI(data)
    print(f"Sygnały {s_str} z modelu RSI:\n{(model.Decision()==s).sum()}")
    
    print()

Sygnały sell z modelu MA:
NEM.US     14
AU.US      17
RGLD.US    19
dtype: int64


In [ ]:
model.Decision()